### jupyter magic commands
Jupyter provides a number of magic commands, with universal behaviors defined outside RevBayes (and other languages executable within Jupyter). To list the magic commands available in Jupyter:

In [ ]:
%lsmagic

Suppose we simulate and save a `Tree` object to file using `writeNexus` in RevBayes. We can then open that tree file from within the same Juptyer notebook using the `%python` magic command.

First, create and save the `Tree` object

In [ ]:
n_taxa = 10
for (i in 1:n_taxa) {
    taxa[i] = taxon("T"+i, 0.0)
}
phy ~ dnBDP(lambda=1, rootAge=1, taxa=taxa)
writeNexus(data=phy, filename="example/output/magic_test.tre")

### Executing `python` code
The magic command `%matplotlib inline` is supposed to, unsurprisingly, plot figures inline. Needs further testing to work with `Phylo.draw()`.

In [1]:
%matplotlib MacOSX

Use `%%python` to indicate the next cell conatins python commands

In [ ]:
%%python

# python stuff
import dendropy as dp
import matplotlib
from Bio import Phylo

# read a tree in Dendropy
dtree = dp.Tree.get(path='example/output/magic_test.tre',schema='nexus')

# read a tree in Biopython
bptree = Phylo.read('example/output/magic_test.tre', 'nexus')

# plot something
Phylo.draw(bptree)

### Executing `R` code through `rpy2`
This is under development, but we can potentially use the `rpy2` package to execute `R` commands through `python`.
Unfortunately, it appears that magic command wrappers must be written per language; they don't exist for `R`, suggesting it's a bit of work.

In [ ]:
%%python
# testing to see if we can pipe into R through rpy2
# R stuff
from rpy2.robjects import r
import tempfile
def to_ape(tree):
    """Convert a tree to the type used by the R package `ape`, via rpy2.

    Requirements:
        - Python package `rpy2`
        - R package `ape`
    """
    with tempfile.NamedTemporaryFile() as tmpf:
        Phylo.write(tree, tmpf, 'newick')
        tmpf.flush()
        rtree = r("""
            library('ape')
            read.tree('%s')
            """ % tmpf.name)
    return rtree


# read a tree in R through rpy
rtree = to_ape(bptree)